In [21]:
import pandas as pd
import json
from datetime import timedelta

In [2]:
# read in bussiness json, subset to only PA
rest_data = []
with open("data/yelp_dataset/yelp_academic_dataset_business.json", 'r') as data_file:
    for line in data_file:
        rest_data.append(json.loads(line))

rest_df = pd.DataFrame(rest_data)
rest_df
pa_restuarants = rest_df[rest_df['state'] == 'PA']

In [22]:
# inspections data
# downloaded from here: https://data.pa.gov/Public-Safety/Public-Food-Inspections-last-24-months-County-Agri/etb6-jzdg/about_data
phila_inspecs = pd.read_csv('data/phila/phila_inspections.csv')

# drop rows with missing labels
inspections = phila_inspecs.dropna(subset=['Overall Compliance'])
inspections

,Organization Name,Active Indicator,Public Facility Name,Program Group Type,Address,City,County Name,Zip Code,State,Inspection Date,Inspection Reason Type,Overall Compliance,Georeferenced Latitude and Longitude
3,Allentown City,Yes,BEERS TO GO,Food,1901 HAMILTON ST,Allentown,Lehigh,18104.0,PA,01/08/2020,Regular,Yes,NaN
7,Allentown City,Yes,SALVATORE RUFFINO PIZZA,Food,1902 ALLEN ST,Allentown,Lehigh,18104.0,PA,03/16/2018,Regular,Yes,NaN
8,Allentown City,Yes,9TH & LINDEN DINER,Food,845 LINDEN ST,Allentown,Lehigh,18101.0,PA,01/17/2019,Opening,Yes,NaN
17,Chambersburg Borough,Yes,"R.C. HERR, LLC",Food,Special Events,Chambersburg,Franklin,17201.0,PA,10/19/2019,Regular,Yes,NaN
35,Emmaus Borough,Yes,TOP STAR EXPRESS,Food,14 Main ST,Emmaus,Lehigh,18049.0,PA,08/01/2019,Regular,Yes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112054,PENNSYLVANIA DEPARTMENT OF AGRICULTURE,Yes,COCALICO COMMUNITY CHURCH,Food,1200 W SWARTZVILLE RD,Reinholds,Lancaster,17569.0,PA,02/14/2019,Regular,Yes,POINT (-76.105947 40.265063)
112055,PENNSYLVANIA DEPARTMENT OF AGRICULTURE,Yes,ARTIC ACE,Food,626 E MAIN ST,Bath,Northampton,18014.0,PA,07/17/2019,Regular,Yes,POINT (-75.381995 40.722597)
112056,PENNSYLVANIA DEPARTMENT OF AGRICULTURE,Yes,J. GUMBO'S,Food,10 St. Francis Way,Cranberry Township,Butler,16066.0,PA,11/21/2019,Regular,No,POINT (-80.099907 40.682181)
112058,PENNSYLVANIA DEPARTMENT OF AGRICULTURE,Yes,WALGREENS #10095,Food,1000 LINCOLN PL,GREENSBURG,Westmoreland,15601.0,PA,04/05/2018,Regular,Yes,POINT (-79.578381 40.302705)


In [5]:
# pre-merge cleaning
inspections.loc[:, 'Address'] = inspections.loc[:, 'Address'].astype(str)
inspections.loc[:, 'Address'] = inspections.loc[:, 'Address'].str.strip()
inspections.loc[:, 'Address'] = inspections.loc[:, 'Address'].str.upper()

pa_restuarants.loc[:, 'address'] = pa_restuarants.loc[:, 'address'].astype(str)
pa_restuarants.loc[:, 'address'] = pa_restuarants.loc[:, 'address'].str.strip()
pa_restuarants.loc[:, 'address'] = pa_restuarants.loc[:, 'address'].str.upper()

Total inspections 112061, when merge on first address then drop to 86140 observations.

In [6]:
labeled_inspections = pd.merge(inspections, pa_restuarants, how='left', left_on='Address', right_on='address').dropna()
labeled_inspections['Overall Compliance'].value_counts()

Overall Compliance
Yes    5417
No     1298
Name: count, dtype: int64

In [9]:
# get lists of resturant ids that have been inspected
inspected_restaurant_ids = list(labeled_inspections.loc[labeled_inspections['Overall Compliance'].notna(), 'business_id'])

In [8]:
# read in review text data
review_data = []
with open("data/yelp_dataset/yelp_academic_dataset_review.json", 'r') as data_file:
    for line in data_file:
        review_data.append(json.loads(line))
review_df = pd.DataFrame(review_data)

In [10]:
# only keep reviews that are for restaurants in our inspected dataset
reviews_for_inspected_restaurants = review_df[review_df.loc[:,'business_id'].isin(inspected_restaurant_ids)]

#convert date to datetime
reviews_for_inspected_restaurants.loc[:, 'date'] = pd.to_datetime(reviews_for_inspected_restaurants.loc[:,'date'])

In [117]:
#get min and max dates in inspection data
labeled_inspections.loc[:, "Inspection Date"] = pd.to_datetime(labeled_inspections.loc[:,"Inspection Date"])
min_labeled_inspections_date = min(labeled_inspections.loc[:,"Inspection Date"])
max_labeled_inspections_date = max(labeled_inspections.loc[:,"Inspection Date"])

#subset reviews for only the ones within viable inspection periods
reviews_for_inspected_restaurants = reviews_for_inspected_restaurants.loc[
    (reviews_for_inspected_restaurants.loc[:,'date']  >= (min_labeled_inspections_date - timedelta(days=12*30))) &
    (reviews_for_inspected_restaurants.loc[:,'date']  <= max_labeled_inspections_date),:]

In [118]:
labeled_inspections = labeled_inspections.sort_values(by=['Public Facility Name',"Inspection Date"], ascending=[True,True])
labeled_inspections["prev_date"] = labeled_inspections.groupby(['Public Facility Name'])["Inspection Date"].shift()

# group all reviews within an inspection period (last inspection date or 6 months before if no last inspection)
labeled_inspections["prev_date_with_nulls"] = labeled_inspections["prev_date"]
labeled_inspections.loc[labeled_inspections["prev_date"].isna(),"prev_date_with_nulls"] = labeled_inspections["Inspection Date"] - timedelta(12*30)

In [119]:
reviews_for_inspected_restaurants #.groupby("business_id").count()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
38,JBWZmBy69VMggxj3eYn17Q,aFa96pz67TwOFu4Weq5Agg,kq5Ghhh14r-eCxlVmlyd8w,5.0,0,0,0,My boyfriend and I tried this deli for the fir...,2018-08-23 21:39:38
1369,GbMcGLNuFFAEx38nUkxkgQ,EF6cPce_wRK8pLczMJwJcA,cX1VH_vpqGoLrio4sV9xcA,4.0,0,0,0,Rons still a regular stop for me. Usually som...,2018-03-04 02:37:38
1657,5cOpGZhbkyd_2XeBTLr56w,Qnxw1ntM_EmrjkQgPXAZ5g,UuWNE12-HI-4UPilREGX5g,5.0,0,0,0,"Studying previous yelp reviews, we decided to...",2018-04-04 01:36:22
2689,S-IwFgSGRkGEG8U89K7f4A,uvqiIB2tmBEpB7kPDJnivw,lTlii3QHEVvsagPGsCtu9w,1.0,1,0,0,I've been eating here when I'm in town for abo...,2018-02-23 22:13:25
3391,A-G9WevyHj9zHjKQafE4AQ,ELo9YjYCw2kIEvJFggN09A,GqUv8rF5h3ZZFfj7S9tgyQ,3.0,0,0,0,I've stopped here twice on a Sunday for the lu...,2018-02-11 18:34:12
...,...,...,...,...,...,...,...,...,...
6988798,fg-TxoZVs5HvrM_bPJ83FA,VyU5wZeQeijq1HRbu5UEXw,B2xtWMBTyDtd-ndqqaIC1Q,3.0,2,0,0,After eating across the street at Fogo de Chão...,2019-09-17 19:22:07
6989252,mqSo1BAaFHxRT9VIgbPulw,zV99rzBLsU5KUWd6oJhc-w,js8ZrAG63h81Lt83wweSwg,5.0,1,0,0,Love this place! Easy to get to. Laid back. Pl...,2020-01-18 05:35:54
6989639,XmeXeK-EG5Ghw8TeEIl5Rw,4ubBKMNRphEtRWX4sZ4b1w,VhupSRgSHupX87HcRNAcHg,5.0,1,0,1,My dad and I just tried this place for the fir...,2018-08-13 15:12:28
6989827,5W8WkJUVo1pgigMYNfzIFA,XT8U_rCZ4c15BoFkDjkOIA,dgEvzJHBA35xZG2DYfFdaQ,1.0,6,1,0,As soon as we approached the hostess station o...,2019-10-21 13:45:43


In [120]:
labeled_inspections

,Organization Name,Active Indicator,Public Facility Name,Program Group Type,Address,City,County Name,Zip Code,State,Inspection Date,...,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,prev_date,prev_date_with_nulls
10735,Chester County Health Department,Yes,"158 LICENSE, LLC",ChesterCountyFood,158 W GAY ST,West Chester,Chester,19380.0,PA,2019-03-20 00:00:00,...,39.959266,-75.607422,2.5,116.0,0.0,"{'RestaurantsPriceRange2': '2', 'RestaurantsAt...","Restaurants, American (New)","{'Monday': '11:0-2:0', 'Tuesday': '11:0-2:0', ...",NaN,2018-03-25 00:00:00
36357,PENNSYLVANIA DEPARTMENT OF AGRICULTURE,Yes,201 WEST FOOD & SPIRITS,Food,201 W LANCASTER AVE,SHILLINGTON,Berks,19607.0,PA,2019-05-03 00:00:00,...,40.046001,-75.424019,1.5,6.0,1.0,"{'BusinessParking': 'None', 'RestaurantsDelive...","Gas Stations, Automotive, Convenience Stores, ...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",NaN,2018-05-08 00:00:00
36358,PENNSYLVANIA DEPARTMENT OF AGRICULTURE,Yes,201 WEST FOOD & SPIRITS,Food,201 W LANCASTER AVE,SHILLINGTON,Berks,19607.0,PA,2019-05-03 00:00:00,...,40.044080,-75.384397,4.0,17.0,0.0,"{'BusinessParking': '{'garage': False, 'street...","Shopping, Accessories, Fashion, Women's Clothing","{'Monday': '0:0-0:0', 'Tuesday': '10:0-20:0', ...",2019-05-03 00:00:00,2019-05-03 00:00:00
56848,PENNSYLVANIA DEPARTMENT OF AGRICULTURE,Yes,3 B'S LOUNGE,Food,309 S MAIN ST,ZELIENOPLE,Butler,16063.0,PA,2018-05-16 00:00:00,...,40.353414,-75.305458,4.0,29.0,1.0,"{'RestaurantsGoodForGroups': 'True', 'Business...","American (Traditional), Bars, Nightlife, Sandw...","{'Monday': '11:30-0:0', 'Tuesday': '11:30-0:0'...",NaN,2017-05-21 00:00:00
72731,PENNSYLVANIA DEPARTMENT OF AGRICULTURE,Yes,3 B'S LOUNGE,Food,309 S MAIN ST,ZELIENOPLE,Butler,16063.0,PA,2019-06-03 00:00:00,...,40.353414,-75.305458,4.0,29.0,1.0,"{'RestaurantsGoodForGroups': 'True', 'Business...","American (Traditional), Bars, Nightlife, Sandw...","{'Monday': '11:30-0:0', 'Tuesday': '11:30-0:0'...",2018-05-16 00:00:00,2018-05-16 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7367,Chester County Health Department,Yes,YORI'S CHURCH STREET BAKERY,ChesterCountyFood,15 N CHURCH ST,West Chester,Chester,19380.0,PA,2018-06-28 00:00:00,...,39.959540,-75.605789,4.5,86.0,1.0,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Restaurants, Breakfast & Brunch, Food, Dessert...","{'Monday': '7:0-19:0', 'Tuesday': '7:0-19:0', ...",NaN,2017-07-03 00:00:00
4300,Chester County Health Department,Yes,YORI'S CHURCH STREET BAKERY,ChesterCountyFood,15 N CHURCH ST,West Chester,Chester,19380.0,PA,2019-07-01 00:00:00,...,39.959540,-75.605789,4.5,86.0,1.0,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Restaurants, Breakfast & Brunch, Food, Dessert...","{'Monday': '7:0-19:0', 'Tuesday': '7:0-19:0', ...",2018-06-28 00:00:00,2018-06-28 00:00:00
10912,Chester County Health Department,Yes,ZWEI RESTAURANT LLC,ChesterCountyFood,163 W LINCOLN HWY,Exton,Chester,19341.0,PA,2018-08-07 00:00:00,...,40.030649,-75.631668,3.5,72.0,1.0,"{'Alcohol': 'u'none'', 'OutdoorSeating': 'Fals...","Sushi Bars, Restaurants, Thai, Chinese","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",NaN,2017-08-12 00:00:00
13958,Chester County Health Department,Yes,ZWEI RESTAURANT LLC,ChesterCountyFood,163 W LINCOLN HWY,Exton,Chester,19341.0,PA,2018-08-23 00:00:00,...,40.030649,-75.631668,3.5,72.0,1.0,"{'Alcohol': 'u'none'', 'OutdoorSeating': 'Fals...","Sushi Bars, Restaurants, Thai, Chinese","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",2018-08-07 00:00:00,2018-08-07 00:00:00


In [137]:
# join design
def extract_relevant_reviews(row):
    #filter all reviews down to only those relevant for the inspection
    relevant_reviews = reviews_for_inspected_restaurants.loc[(reviews_for_inspected_restaurants.loc[:,"business_id"] == row["business_id"]) &
                                                            (reviews_for_inspected_restaurants.loc[:,"date"] >= row["prev_date_with_nulls"]) &
                                                            (reviews_for_inspected_restaurants.loc[:,"date"] <= row["Inspection Date"]),:]
    #return both the list of text reviews and the stars given (if needed)
    n_reviews=len(list(relevant_reviews["stars"]))
    
    if len(list(relevant_reviews["stars"])) > 0:
        avg_rating = sum(list(relevant_reviews["stars"]))/len(list(relevant_reviews["stars"]))
    else:
        avg_rating = None

    return list(relevant_reviews["text"]), list(relevant_reviews["stars"]), n_reviews, avg_rating


extract_relevant_reviews(labeled_inspections.iloc[0,:])

(["Our first visit here and probably won't be back.  Everything had a very unusual taste. The fried cauliflower was sour as though it was seasoned with vinegar. I ordered nachos....also sour. The chicken was peppery.  Not my fav.",
  "I would give it 3 stars for food and 4 for drinks. I went for happy hour and the drinks were cheap and yummy. The food was okay but nothing special. It was packed during happy hour and there was one bartender. She held her own but that's a lot of people for just her. Also, after it died down I was just hanging with friends and you had to flag down someone for just water even though it was empty\nand not much to do. \nI definitely would come back for drinks and even food but I would never pay full price.",
  "Went to landmark for my boyfriend's birthday for Game of Thrones trivia. I work in a restaurant, so i know it's hectic but our service was god awful. Our waitress came over and gave us menus and our friend ordered an app right away and we ordered drin

In [138]:
#apply the above function to each row of the dataset
sorted_reviews = labeled_inspections.apply(extract_relevant_reviews, axis=1, result_type='expand').rename(columns={0:"reviews",1:"ratings", 2:"n_reviews", 3: "avg_rating"})

In [155]:
labeled_inspections_with_reviews = labeled_inspections.merge(right=sorted_reviews, right_index=True, left_index=True)

* 2094 / 6715 inspection periods have reviews within the period if the "previous inspection if null" window is set to 6 months.
* 2165 / 6715 inspection periods have reviews within the period if the "previous inspection if null" window is set to 12 months.

In [154]:
# total number of used reviews
labeled_inspections_with_reviews.n_reviews.sum()

16897

In [157]:
#filter out observations without reviews
labeled_inspections_with_reviews = labeled_inspections_with_reviews.loc[labeled_inspections_with_reviews.loc[:,"n_reviews"] > 0,:]
labeled_inspections_with_reviews.to_csv("data/phila/labeled_inspections_with_reviews.csv")

In [158]:
# total number of unique restaurants
len(labeled_inspections_with_reviews.business_id.unique())

838